In [3]:
import numpy as np 
import pandas as pd 

from keras.models import Sequential 
from keras.layers import Dense 
from keras.utils import to_categorical 
from keras.datasets import fashion_mnist 
from keras.metrics import Recall, Precision 

import matplotlib.pyplot as plt

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

train_images = (train_images / 127) - 1
test_images = (test_images / 127) - 1

train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

y_t = to_categorical(train_labels)
y_tt = to_categorical(test_labels)

4423680/4422102 [==============================] - 1s 0us/step


### Разное количество нейронов в одном слое

In [5]:
res_list = []
for i in range(0, 12):

  result = {}
  model_neurons = Sequential([
      Dense(2**i, activation='relu', input_shape=(784,)),
      Dense(10, activation='softmax')
  ])

  model_neurons.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  model_neurons.fit(
      train_images,
      to_categorical(train_labels),
      epochs=5,
      batch_size=32, 
      validation_split=0.2,
      verbose=0
  )

  result_train = model_neurons.evaluate(train_images, to_categorical(train_labels), verbose=0)
  result_test = model_neurons.evaluate(test_images, to_categorical(test_labels), verbose=0)

  result['n_neurons'] = 2**i
  result['acc_train'] = np.round(result_train[1], 4)
  result['acc_test'] = np.round(result_test[1], 4)

  res_list.append(result)

In [6]:
from pprint import pprint
pprint(res_list)

[{'acc_test': 0.3676, 'acc_train': 0.3735, 'n_neurons': 1},
 {'acc_test': 0.6522, 'acc_train': 0.653, 'n_neurons': 2},
 {'acc_test': 0.7788, 'acc_train': 0.7936, 'n_neurons': 4},
 {'acc_test': 0.829, 'acc_train': 0.8475, 'n_neurons': 8},
 {'acc_test': 0.8514, 'acc_train': 0.8711, 'n_neurons': 16},
 {'acc_test': 0.8532, 'acc_train': 0.8731, 'n_neurons': 32},
 {'acc_test': 0.8585, 'acc_train': 0.8813, 'n_neurons': 64},
 {'acc_test': 0.87, 'acc_train': 0.8978, 'n_neurons': 128},
 {'acc_test': 0.8686, 'acc_train': 0.8956, 'n_neurons': 256},
 {'acc_test': 0.8556, 'acc_train': 0.8818, 'n_neurons': 512},
 {'acc_test': 0.8651, 'acc_train': 0.8929, 'n_neurons': 1024},
 {'acc_test': 0.8769, 'acc_train': 0.9018, 'n_neurons': 2048}]


С увеличением колличества нейронов accuracy растёт, затем начинает падать. Лучший результат на 128 нейронах , и сопастовимы результат на 2048 но тут более сильное переобучение

### Разное количество слоёв с 256 нейронов.

In [7]:
model_2l = Sequential([
      Dense(256, activation='relu', input_shape=(784,)), 
      Dense(10, activation='softmax')
  ])

model_3l = Sequential([
      Dense(256, activation='relu', input_shape=(784,)),
      Dense(256, activation='relu'),
      Dense(10, activation='softmax')
  ]) 

model_5l = Sequential([
      Dense(256, activation='relu', input_shape=(784,)),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(10, activation='softmax')
  ]) 

model_10l = Sequential([
      Dense(256, activation='relu', input_shape=(784,)),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(10, activation='softmax')
  ])

In [8]:
res_list = []
l = [2, 3, 5, 10]
i = 0
for model in (model_2l, model_3l, model_5l, model_10l):

  result = {}

  model.summary()

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  model.fit(
      train_images,
      to_categorical(train_labels),
      epochs=5,
      batch_size=32, 
      validation_split=0.2,
      verbose=0
  )

  result_train = model.evaluate(train_images, to_categorical(train_labels), verbose=0)
  result_test = model.evaluate(test_images, to_categorical(test_labels), verbose=0)

  result['n_layers'] = l[i]
  i += 1
  result['acc_train'] = np.round(result_train[1], 4)
  result['acc_test'] = np.round(result_test[1], 4)

  res_list.append(result)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_25 (Dense)             (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_27 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_28 (Dense)             (None, 10)                2570      
Total params: 269,322
Tr

In [9]:
pprint(res_list)

[{'acc_test': 0.8673, 'acc_train': 0.8938, 'n_layers': 2},
 {'acc_test': 0.8709, 'acc_train': 0.8978, 'n_layers': 3},
 {'acc_test': 0.8683, 'acc_train': 0.8927, 'n_layers': 5},
 {'acc_test': 0.8472, 'acc_train': 0.8665, 'n_layers': 10}]


самый лучшый результат на 2-3 слоях дальше accuracy уменьшается 

### Разное количество слоёв, примерно одно и тоже количество параметров

In [11]:
model_2l = Sequential([
      Dense(256, activation='relu', input_shape=(784,)), 
      Dense(10, activation='softmax')
  ])

model_3l = Sequential([
      Dense(222, activation='relu', input_shape=(784,)),
      Dense(222, activation='relu'),
      Dense(10, activation='softmax')
  ]) 

model_5l = Sequential([
      Dense(164, activation='relu', input_shape=(784,)),
      Dense(164, activation='relu'),
      Dense(164, activation='relu'),
      Dense(164, activation='relu'),
      Dense(10, activation='softmax')
  ]) 

model_10l = Sequential([
      Dense(128, activation='relu', input_shape=(784,)),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(10, activation='softmax')
  ])

In [12]:
res_list = []
l = [2, 3, 5, 10]
i = 0
for model in (model_2l, model_3l, model_5l, model_10l):

  result = {}

  model.summary()

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  model.fit(
      train_images,
      to_categorical(train_labels),
      epochs=5,
      batch_size=32, 
      validation_split=0.2,
      verbose=0
  )

  result_train = model.evaluate(train_images, to_categorical(train_labels), verbose=0)
  result_test = model.evaluate(test_images, to_categorical(test_labels), verbose=0)

  result['n_layers'] = l[i]
  i += 1
  result['acc_train'] = np.round(result_train[1], 4)
  result['acc_test'] = np.round(result_test[1], 4)

  res_list.append(result)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_45 (Dense)             (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 222)               174270    
_________________________________________________________________
dense_47 (Dense)             (None, 222)               49506     
_________________________________________________________________
dense_48 (Dense)             (None, 10)                2230      
Total params: 226,006
Tr

In [13]:
pprint(res_list)

[{'acc_test': 0.87, 'acc_train': 0.8942, 'n_layers': 2},
 {'acc_test': 0.8682, 'acc_train': 0.8976, 'n_layers': 3},
 {'acc_test': 0.8689, 'acc_train': 0.8903, 'n_layers': 5},
 {'acc_test': 0.8557, 'acc_train': 0.8753, 'n_layers': 10}]


Лучший результат на 2-5 слоях, с увеличением колличества слоев accuracy становится меньше 

### Тестируем разные оптимизаторы

In [15]:
res_list = []

for opt in ['SGD', 'Adam', 'RMSProp']:

  result = {}

  model_opti = Sequential([
      Dense(256, activation='relu', input_shape=(784,)), 
      Dense(10, activation='softmax')
  ])

  model_opti.compile(
      optimizer=opt,
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  model_opti.fit(
      train_images,
      to_categorical(train_labels),
      epochs=5,
      batch_size=32, 
      validation_split=0.2,
      verbose=0
  )

  result_train = model_opti.evaluate(train_images, to_categorical(train_labels), verbose=0)
  result_test = model_opti.evaluate(test_images, to_categorical(test_labels), verbose=0)

  result['opt'] = opt
  i += 1
  result['acc_train'] = np.round(result_train[1], 4)
  result['acc_test'] = np.round(result_test[1], 4)

  res_list.append(result)

In [16]:
pprint(res_list)

[{'acc_test': 0.853, 'acc_train': 0.8697, 'opt': 'SGD'},
 {'acc_test': 0.8732, 'acc_train': 0.8998, 'opt': 'Adam'},
 {'acc_test': 0.8651, 'acc_train': 0.8914, 'opt': 'RMSProp'}]


Adam справляет чуть лучше RMSProp

### Классы

In [17]:
model_opti = Sequential([
      Dense(256, activation='relu', input_shape=(784,)), 
      Dense(10, activation='softmax')
  ])

model_opti.compile(
      optimizer=opt,
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

model_opti.fit(
      train_images,
      to_categorical(train_labels),
      epochs=5,
      batch_size=32, 
      validation_split=0.2,
      verbose=0
  )

In [18]:
predictions = model_2l.predict(test_images)

# Print our model's predictions.
test_labels_predict = np.argmax(predictions, axis=1)

In [19]:
labels = np.unique(train_labels)
df = pd.DataFrame(columns=['recall', 'precision'], index=range(10))

for i in labels:
  recall = Recall()
  recall.update_state(y_true=(test_labels == i), 
                      y_pred=(test_labels_predict == i))
  df.loc[i, 'recall'] = recall.result().numpy()

  precision = Precision()
  precision.update_state(y_true=(test_labels == i), 
                         y_pred=(test_labels_predict == i))
  df.loc[i, 'precision'] = precision.result().numpy()

  df.loc[i, 'F'] = 2*recall.result().numpy()*precision.result().numpy()/(precision.result().numpy()+recall.result().numpy())

df.index = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

df

,recall,precision,F
T-shirt/top,0.831,0.798271,0.814307
Trouser,0.976,0.956863,0.966337
Pullover,0.745,0.830546,0.785451
Dress,0.818,0.921171,0.866525
Coat,0.851,0.773636,0.810476
Sandal,0.919,0.976621,0.946935
Shirt,0.688,0.655863,0.671547
Sneaker,0.954,0.908571,0.930732
Bag,0.962,0.969758,0.965863
Ankle boot,0.956,0.935421,0.945598
